In [2]:
# Connect to MetaTrader 5 and login
import MetaTrader5 as mt5

# Initialize connection
if not mt5.initialize():
    print("Failed to initialize MT5")
else:
    # Login to account
    authorized = mt5.login(
        11101038, password="x*x^JM8u", server="VantageInternational-Demo"
    )
    if authorized:
        account_info = mt5.account_info()
        if account_info is not None:
            print(f"Account #: {account_info.login}")
            print(f"Leverage: {account_info.leverage}")
            print(f"Balance: {account_info.balance}")
            print(f"Equity: {account_info.equity}")
            print(f"Margin: {account_info.margin}")
        else:
            print("Failed to retrieve account info")
    else:
        print("Failed to login to account")

Account #: 11101038
Leverage: 100
Balance: 2487.03
Equity: 2487.03
Margin: 0.0


In [23]:
# Get OHLC data for GBPUSD from start_date until now
from datetime import datetime
import polars as pl
import pytz

# Set date range with timezone (MT5 uses UTC)
timezone = pytz.timezone("Etc/UTC")
start_date = datetime(2025, 10, 1, tzinfo=timezone)
end_date = datetime.now(timezone)

# Fetch OHLC data from MT5 (M1 = 1-minute timeframe)
rates = mt5.copy_rates_range("GBPUSD", mt5.TIMEFRAME_M15, start_date, end_date)

# Check if data was retrieved successfully
if rates is None or len(rates) == 0:
    print(f"Failed to retrieve OHLC data. Error: {mt5.last_error()}")
else:
    # Convert to Polars DataFrame
    df = pl.DataFrame(rates)

    # Add index column and convert time to datetime
    df = df.with_columns(
        pl.int_range(0, pl.len()).alias("index"),
        pl.from_epoch(pl.col("time"), time_unit="s").alias("time"),
    )

df

time,open,high,low,close,tick_volume,spread,real_volume,index
datetime[μs],f64,f64,f64,f64,u64,i32,u64,i64
2025-10-01 00:00:00,1.34436,1.34436,1.34392,1.34399,18,75,0,0
2025-10-01 00:15:00,1.34399,1.34406,1.3439,1.34396,38,73,0,1
2025-10-01 00:30:00,1.34396,1.34429,1.34391,1.34404,691,67,0,2
2025-10-01 00:45:00,1.34404,1.34428,1.34359,1.34408,841,67,0,3
2025-10-01 01:00:00,1.34404,1.34515,1.34403,1.34473,492,11,0,4
…,…,…,…,…,…,…,…,…
2025-10-17 22:45:00,1.34294,1.34372,1.34282,1.34371,823,11,0,1243
2025-10-17 23:00:00,1.34371,1.34373,1.34261,1.34294,601,11,0,1244
2025-10-17 23:15:00,1.34294,1.34305,1.34258,1.34264,587,11,0,1245


In [24]:
def get_london_session(hour):
    if 8 <= hour < 16:
        return "london session"
    else:
        return None


df = df.with_columns([pl.col("time").dt.hour().alias("hour")])

df = df.with_columns(
    pl.when((pl.col("hour") >= 8) & (pl.col("hour") < 16))
    .then(pl.lit("london session"))
    .otherwise(None)
    .alias("session")
)

df = df.with_columns([pl.col("time").dt.date().alias("date")])

df

time,open,high,low,close,tick_volume,spread,real_volume,index,hour,session,date
datetime[μs],f64,f64,f64,f64,u64,i32,u64,i64,i8,str,date
2025-10-01 00:00:00,1.34436,1.34436,1.34392,1.34399,18,75,0,0,0,null,2025-10-01
2025-10-01 00:15:00,1.34399,1.34406,1.3439,1.34396,38,73,0,1,0,null,2025-10-01
2025-10-01 00:30:00,1.34396,1.34429,1.34391,1.34404,691,67,0,2,0,null,2025-10-01
2025-10-01 00:45:00,1.34404,1.34428,1.34359,1.34408,841,67,0,3,0,null,2025-10-01
2025-10-01 01:00:00,1.34404,1.34515,1.34403,1.34473,492,11,0,4,1,null,2025-10-01
…,…,…,…,…,…,…,…,…,…,…,…
2025-10-17 22:45:00,1.34294,1.34372,1.34282,1.34371,823,11,0,1243,22,null,2025-10-17
2025-10-17 23:00:00,1.34371,1.34373,1.34261,1.34294,601,11,0,1244,23,null,2025-10-17
2025-10-17 23:15:00,1.34294,1.34305,1.34258,1.34264,587,11,0,1245,23,null,2025-10-17


In [25]:
# Aggregate by date to get daily high/low from OHLC data
df_by_date = df.group_by(['date', 'session'], maintain_order=True).agg(
    pl.col('high').max().alias('session_high'),
    pl.col('low').min().alias('session_low'),
)
df_by_date

date,session,session_high,session_low
date,str,f64,f64
2025-10-01,null,1.3527,1.3434
2025-10-01,"""london session""",1.35213,1.3453
2025-10-02,null,1.34868,1.34003
2025-10-02,"""london session""",1.35097,1.34632
2025-10-03,null,1.34858,1.34303
…,…,…,…
2025-10-15,"""london session""",1.33728,1.33356
2025-10-16,null,1.3454,1.33863
2025-10-16,"""london session""",1.34486,1.34036


In [26]:
df = df.join(df_by_date, on=['date'], how='left')
df

time,open,high,low,close,tick_volume,spread,real_volume,index,hour,session,date,session_right,session_high,session_low
datetime[μs],f64,f64,f64,f64,u64,i32,u64,i64,i8,str,date,str,f64,f64
2025-10-01 00:00:00,1.34436,1.34436,1.34392,1.34399,18,75,0,0,0,null,2025-10-01,null,1.3527,1.3434
2025-10-01 00:00:00,1.34436,1.34436,1.34392,1.34399,18,75,0,0,0,null,2025-10-01,"""london session""",1.35213,1.3453
2025-10-01 00:15:00,1.34399,1.34406,1.3439,1.34396,38,73,0,1,0,null,2025-10-01,null,1.3527,1.3434
2025-10-01 00:15:00,1.34399,1.34406,1.3439,1.34396,38,73,0,1,0,null,2025-10-01,"""london session""",1.35213,1.3453
2025-10-01 00:30:00,1.34396,1.34429,1.34391,1.34404,691,67,0,2,0,null,2025-10-01,null,1.3527,1.3434
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-10-17 23:15:00,1.34294,1.34305,1.34258,1.34264,587,11,0,1245,23,null,2025-10-17,"""london session""",1.34711,1.34078
2025-10-17 23:30:00,1.34264,1.3428,1.34197,1.34267,562,11,0,1246,23,null,2025-10-17,null,1.34533,1.33904
2025-10-17 23:30:00,1.34264,1.3428,1.34197,1.34267,562,11,0,1246,23,null,2025-10-17,"""london session""",1.34711,1.34078


In [31]:
# Calculate stoploss as the midpoint between session high and low
df = df.with_columns([
    ((pl.col('session_high') + pl.col('session_low')) / 2).alias('stoploss')
])

df[['time', 'session_high', 'session_low', 'stoploss']].head(10)

time,session_high,session_low,stoploss
datetime[μs],f64,f64,f64
2025-10-01 00:00:00,1.3527,1.3434,1.34805
2025-10-01 00:00:00,1.35213,1.3453,1.348715
2025-10-01 00:15:00,1.3527,1.3434,1.34805
2025-10-01 00:15:00,1.35213,1.3453,1.348715
2025-10-01 00:30:00,1.3527,1.3434,1.34805
2025-10-01 00:30:00,1.35213,1.3453,1.348715
2025-10-01 00:45:00,1.3527,1.3434,1.34805
2025-10-01 00:45:00,1.35213,1.3453,1.348715
2025-10-01 01:00:00,1.3527,1.3434,1.34805


In [34]:
# Plot OHLC candlestick chart
import plotly.graph_objects as go

fig = go.Figure()

# Add candlestick chart
fig.add_trace(
    go.Candlestick(
        x=df["time"],
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
        name="OHLC",
    )
)

fig.update_layout(
    title="GBPUSD OHLC Prices",
    xaxis_title="Time",
    yaxis_title="Price",
    hovermode="x unified",
    template="plotly_dark",
)

# Add session_high line
fig.add_trace(
    go.Scatter(
        x=df["time"],
        y=df["session_high"],
        mode="lines",
        name="Session High",
        line=dict(color="green", width=0.5),
        fill=None,
    )
)

# Add session_low line
fig.add_trace(
    go.Scatter(
        x=df["time"],
        y=df["session_low"],
        mode="lines",
        name="Session Low",
        line=dict(color="orange", width=0.5),
        fill=None,
    )
)

# Add stoploss line
fig.add_trace(
    go.Scatter(
        x=df["time"],
        y=df["stoploss"],
        mode="lines",
        name="Stoploss",
        line=dict(color="red", width=0.5),
        fill=None,
    )
)

fig.show()